## Data Preparation

In [1]:
import pandas as pd
import re

def check_title(title):
    slash_cnt = title.count('/')
    comma_cnt = title.count(',')
    
    if slash_cnt > 1 or comma_cnt > 1: 
        return 0
    
    else:
        return 1

def clean_title(title):
    title = title.lower()
    title = re.sub('gx','g x ',title)
    title = re.sub('[0-9]+',' ',title)
    title = re.sub('[\[\]()!?_~#!@$%^&*-+=<>,\"\'/]',' ',title)
    split_title = title.split(' ')
    title = " ".join([i for i in split_title if len(i) > 0])
    
    return title.strip()
    
title_df = pd.read_csv('data/가공식품_title_data.csv',  usecols = ['prodNm'], encoding='euc-kr')
title_df['label'] = title_df['prodNm'].apply(lambda x : check_title(x))
title_df['cleaned_prodNm'] = title_df['prodNm'].apply(lambda x : clean_title(x))
title_df = title_df.drop_duplicates().reset_index(drop=True)

print(title_df.groupby(['label']).count())

# make train data 
false_data = title_df[(title_df['label'] == 0) & (~title_df['cleaned_prodNm'].str.contains('pqm'))]
true_data = title_df[(title_df['label'] == 1) & (~title_df['cleaned_prodNm'].str.contains('pqm'))].sample(n = len(false_data), random_state=1)

train_df = pd.concat([false_data, true_data])
train_df.reset_index(drop=True, inplace=True)
train_df = train_df[['prodNm','cleaned_prodNm','label']]
train_df.head()

       prodNm  cleaned_prodNm
label                        
0        7132            7132
1      153667          153667


,prodNm,cleaned_prodNm,label
0,로얄단스크_버터쿠키/오트밀쿠키1kg/수입과자/간식,로얄단스크 버터쿠키 오트밀쿠키 kg 수입과자 간식,0
1,삼립 누네띠네12g 100입/사과맛쿠키/미니 꿀약과/삼립식품/과자,삼립 누네띠네 g 입 사과맛쿠키 미니 꿀약과 삼립식품 과자,0
2,식단조절 건강도시락 6종 6개 /칼로리 다운/,식단조절 건강도시락 종 개 칼로리 다운,0
3,들기름1.75리터/수입들깨100프로/업소기름/식당참기름,들기름 . 리터 수입들깨 프로 업소기름 식당참기름,0
4,식단조절 건강도시락 6종6개 /가벼운칼로리/여름준비/점심도시락,식단조절 건강도시락 종 개 가벼운칼로리 여름준비 점심도시락,0


In [2]:
# save file
train_df.to_csv('data/tarin_prodNm.csv',encoding='euc-kr', index=False)

In [3]:
train_df.head()

,prodNm,cleaned_prodNm,label
0,로얄단스크_버터쿠키/오트밀쿠키1kg/수입과자/간식,로얄단스크 버터쿠키 오트밀쿠키 kg 수입과자 간식,0
1,삼립 누네띠네12g 100입/사과맛쿠키/미니 꿀약과/삼립식품/과자,삼립 누네띠네 g 입 사과맛쿠키 미니 꿀약과 삼립식품 과자,0
2,식단조절 건강도시락 6종 6개 /칼로리 다운/,식단조절 건강도시락 종 개 칼로리 다운,0
3,들기름1.75리터/수입들깨100프로/업소기름/식당참기름,들기름 . 리터 수입들깨 프로 업소기름 식당참기름,0
4,식단조절 건강도시락 6종6개 /가벼운칼로리/여름준비/점심도시락,식단조절 건강도시락 종 개 가벼운칼로리 여름준비 점심도시락,0


## Word2vec trainer

In [11]:
from gensim.models import Word2Vec, KeyedVectors
import params

cleaned_prodNm_list = title_df['cleaned_prodNm'].values.tolist()
sentences = [i.split(' ')for i in cleaned_prodNm_list]

model = Word2Vec(sentences, size=200, window = 5, min_count=1, workers = 16)
model.wv.save_word2vec_format('{0}'.format(params.w2v_path), binary=True)

In [13]:
embed_lookup = KeyedVectors.load_word2vec_format('{0}'.format(params.w2v_path), 
                                                 binary=True)
embed_lookup.vectors.shape

(41721, 200)